In [22]:
#Imports

import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [42]:
#Load the data + data exploration

frame = pd.read_csv("complete-frame.csv", sep=",")
"""
train_x = pd.concat([frame.no_mutations, frame.line_coverage, frame.isAssertionRoulette, frame.isEagerTest, frame.isLazyTest,
frame.isMysteryGuest, frame.isSensitiveEquality, frame.isResourceOptimism, frame.isForTestersOnly,
frame.isIndirectTesting, frame.LOC_prod, frame.HALSTEAD_prod, frame.RFC_prod, frame.CBO_prod, frame.MPC_prod, frame.IFC_prod, frame.DAC_prod,frame.DAC2_prod, frame.LCOM1_prod, frame.LCOM2_prod,
frame.LCOM3_prod, frame.LCOM4_prod, frame.CONNECTIVITY_prod, frame.LCOM5_prod, frame.COH_prod, frame.TCC_prod,
frame.LCC_prod, frame.ICH_prod, frame.WMC_prod, frame.NOA_prod, frame.NOPA_prod, frame.NOP_prod,
frame.McCABE_prod, frame.BUSWEIMER_prod, frame.LOC_test, frame.HALSTEAD_test, frame.RFC_test, frame.CBO_test,
frame.MPC_test, frame.IFC_test, frame.DAC_test, frame.DAC2_test, frame.LCOM1_test, frame.LCOM2_test,
frame.LCOM3_test, frame.LCOM4_test, frame.CONNECTIVITY_test, frame.LCOM5_test, frame.COH_test, frame.TCC_test,
frame.LCC_test, frame.ICH_test, frame.WMC_test, frame.NOA_test, frame.NOPA_test, frame.NOP_test, frame.McCABE_test,
frame.BUSWEIMER_test, frame.csm_CDSBP, frame.csm_CC, frame.csm_FD, frame.csm_Blob, frame.csm_SC, frame.csm_MC,
frame.csm_LM, frame.csm_FE, frame.prod_readability, frame.test_readability], axis=1).round(2)

"""

train_x = pd.concat([frame.no_mutations,
                     #frame.line_coverage,
                     #frame.isEagerTest,
                     frame.LOC_prod, frame.LOC_test, frame.WMC_prod,
                     frame.LCOM1_prod, frame.LCOM2_prod,
                     frame.LCOM4_prod, frame.McCABE_prod,
                     frame.RFC_prod, frame.MPC_prod,
                     frame.RFC_test, frame.MPC_test,
                     frame.LCOM1_test, frame.LCOM2_test,
                     frame.LCOM4_test, frame.LCC_test,
                     frame.LCC_test, frame.WMC_test,
                     frame.McCABE_test, frame.NOP_prod,
                     frame.NOA_prod], axis = 1).round(2)
train_y = pd.concat([frame.mutation], axis = 1).round(1).mul(10).astype(int)
#print(train_y)
#labels = [1,2,3,4,5]
#bins = [0,2,4,6,8,10]
labels = [1,2]
bins = [0,5,10]
frame['mutation_bins'] = pd.cut(frame.mutation.round(1).mul(10), bins=bins, labels = labels, include_lowest=True)
#print(frame.mutation_bins)

train_y = pd.concat([frame.mutation_bins], axis = 1)

train_size = 0.8

train_cnt = math.floor(train_x.shape[0] * train_size)
x_train = train_x.iloc[0:train_cnt].values
y_train = train_y.iloc[0:train_cnt].values
x_test = train_x.iloc[train_cnt:].values
y_test = train_y.iloc[train_cnt:].values

#Divide the remaining data into training data and validation data (for the early stopping)



from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
x_train = min_max_scaler.fit_transform(x_train)
x_test = min_max_scaler.fit_transform(x_test)

# Define the model architecture

model = keras.Sequential()
model.add(keras.layers.Dense(21, input_dim=21, activation=tf.nn.relu))
model.add(keras.layers.Dense(18, activation=tf.nn.relu))
model.add(keras.layers.Dense(12, activation=tf.nn.relu))
model.add(keras.layers.Dense(6, activation=tf.nn.relu))
model.add(keras.layers.Dense(6, activation=tf.nn.softmax))


"""
model = keras.Sequential()
model.add(keras.layers.Dense(68, input_dim=68, activation=tf.nn.relu))
model.add(keras.layers.Dense(50, activation=tf.nn.relu))
model.add(keras.layers.Dense(40, activation=tf.nn.relu))
model.add(keras.layers.Dense(30, activation=tf.nn.relu))
model.add(keras.layers.Dense(20, activation=tf.nn.relu))
model.add(keras.layers.Dense(10, activation=tf.nn.relu))
model.add(keras.layers.Dense(5, activation=tf.nn.relu))
model.add(keras.layers.Dense(2, activation=tf.nn.relu))
"""

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stopping_monitor = keras.callbacks.EarlyStopping(patience=3,restore_best_weights=True)


model.fit(x_train, y_train, epochs=1000, verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[early_stopping_monitor])

# Evaluate the model using test dataset.
test_loss, test_acc = model.evaluate(x_test, y_test)

print('Test accuracy:', test_acc)



Train on 1928 samples, validate on 483 samples
Epoch 1/1000
1928/1928 [==============================] - 1s 538us/sample - loss: 1.6118 - accuracy: 0.5757 - val_loss: 1.0338 - val_accuracy: 0.8737
Epoch 2/1000
1928/1928 [==============================] - 0s 96us/sample - loss: 1.0933 - accuracy: 0.5970 - val_loss: 0.5390 - val_accuracy: 0.8778
Epoch 3/1000
1928/1928 [==============================] - 0s 97us/sample - loss: 0.6920 - accuracy: 0.6432 - val_loss: 0.4325 - val_accuracy: 0.8799
Epoch 4/1000
1928/1928 [==============================] - 0s 96us/sample - loss: 0.6342 - accuracy: 0.6608 - val_loss: 0.4516 - val_accuracy: 0.8613
Epoch 5/1000
1928/1928 [==============================] - 0s 96us/sample - loss: 0.6112 - accuracy: 0.6779 - val_loss: 0.4199 - val_accuracy: 0.8613
Epoch 6/1000
1928/1928 [==============================] - 0s 101us/sample - loss: 0.6102 - accuracy: 0.6774 - val_loss: 0.4218 - val_accuracy: 0.8634
Epoch 7/1000
1928/1928 [==============================] -

Test accuracy: 0.86542445


In [41]:
#Check Overfeet
test_loss, test_acc = model.evaluate(x_train, y_train)



1928/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================